In [2]:
from helper_functions import train, predict
from dataset import train_cats_dogs, test_cats_dogs
import torch
from torch import nn, optim
import torch.nn.functional as F

In [26]:
class ResNet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv1_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, padding='same'),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2_block = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=7, padding='same'),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv64_block = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
        )
        self.conv128_block = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
        )

        self.classification_head = nn.Sequential(
            
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.unsqueeze(dim=0)
        output_conv1 = self.conv1_block(x)
        output_conv2 = self.conv64_block(output_conv1)
        input_conv3 = F.relu(output_conv1 + output_conv2)
        output_conv4 = self.conv64_block(input_conv3)
        input_conv5 = F.relu(input_conv3 + output_conv4)
        output_conv5 = self.conv64_block(input_conv5)
        input_conv6 = F.relu(output_conv5 + input_conv5)
        output_conv7 = self.conv2_block(input_conv6)
        output_conv8 = self.conv128_block(output_conv7)
        output_conv9 = self.conv128_block(self.conv128_block(output_conv8))
        input_conv10 = F.relu(output_conv9 + output_conv8)
        print(input_conv10.shape)


image, labels = next(iter(test_cats_dogs))
model = ResNet()
print(f"Image Shape: {image.shape}")
print(f"Prediction: {predict(model, image)}")

Image Shape: torch.Size([3, 224, 224])
torch.Size([1, 128, 56, 56])
Prediction: None


In [6]:
from torchinfo import summary

summary(model, input_size=(3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"], row_settings=["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
ResNet (ResNet)                          [3, 224, 224]             --                        --                        --
Total params: 0
Trainable params: 0
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0
Input size (MB): 0.60
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.60